https://github.com/dotnet/try/issues/688

In [ ]:
#r "nuget:Microsoft.Data.Analysis"
open Microsoft.Data.Analysis

In [ ]:
#r @"TakensTheorem.Core.dll"

In [ ]:
//#r @"C:\Users\cernu\.nuget\packages\microsoft.data.analysis\0.2.0\lib\netstandard2.0\Microsoft.Data.Analysis.dll"

In [ ]:
open System
open TakensTheorem.Core
open TakensTheorem.Core.ZipHelper

In [4]:
let path = @"C:\Users\cernu\Documents\Datasets\historical-hourly-weather-data.zip"

In [ ]:
let loader = new ZippedCsvLoader(path)

In [ ]:
let cityTable = loader.ToDataFrame "city_attributes.csv"
let temperatureDF = loader.ToDataFrame "temperature.csv"
let indexName = temperatureDF.Columns.[0].Name

In [ ]:
temperatureDF.[indexName] <- temperatureDF.[indexName]
                             |> Seq.cast<string>
                             |> Seq.map (DateTime.Parse)
                             |> DataFrame.AsColumn indexName

In [ ]:
//temperatureDF.["datetime"].ElementwiseGreaterThanOrEqual<DateTime> (start)

In [ ]:
let start = DateTime(2015, 6, 22)
let stop = DateTime(2015, 8, 31)

(* 
    temperatureDF.["datetime"].ElementwiseGreaterThanOrEqual<DateTime> (start)
    don't work for no dates yet
*)
    
let dateFilterColumn = 
    temperatureDF.[indexName]
    |> DataFrameColumn.AsArray<DateTime>
    |> Array.map (fun dt -> dt >= start && dt <= stop)
    |> DataFrame.AsColumn<bool> "dateFilter"
    
let weatherDataMontreal = 
    temperatureDF.Filter(dateFilterColumn).["Montreal"]

In [ ]:
temperatureDF |> DataFrame.Length
weatherDataMontreal.Filter

In [ ]:
// index unavailable at column level

weatherDataMontreal 
|> DataFrameColumn.Plot<float>

In [ ]:
(* subtracting null same as subtracting zero
 * in pandas both become NaN
 * therefore dropping na's after the fact
 * does little to trim the column
 *)
let lowPassFilterSignalCol = weatherDataMontreal.Clone()

weatherDataMontreal
|> DataFrameColumn.Rolling<float> 24 (Seq.average)
|> Seq.iteri (fun i value -> lowPassFilterSignalCol.[int64 (i+12)] <- Nullable value)

let weatherDataMontrealFiltered = 
    weatherDataMontreal - lowPassFilterSignalCol 
    |> DataFrameColumn.Trim<float> 0.0

weatherDataMontrealFiltered
|> DataFrameColumn.Plot<float>

In [ ]:

let embeddedWeather = 
    Takens.Embedding<float> 5 2 weatherDataMontrealFiltered
    |> Array.ofSeq
   
Chart.Plot(Scatter(x=embeddedWeather.[0], y=embeddedWeather.[1]))

In [ ]:
let embeddedWeather3 = 
    Takens.Embedding<float> 6 3 weatherDataMontrealFiltered
    |> Array.ofSeq
    |> to2d

//embeddedWeather3.GetType().Name
Chart.Plot(Scatter3d(x=embeddedWeather3.[0,*], y=embeddedWeather3.[1,*], z=embeddedWeather3.[2,*], mode="lines"))

In [ ]:
(* No drop nulls for columns *)
let l0 = temperatureDF.DropNulls(DropNullOptions.Any).Rows.Count
let ln = temperatureDF.Rows.Count
let l1 = weatherDataMontreal.Length
let l2 = DataFrame(weatherDataMontreal).DropNulls(DropNullOptions.Any).Rows.Count
let l3 = 
    weatherDataMontreal 
    |> DataFrameColumn.AsArray<Nullable<float>>
    |> Seq.filter (fun value -> value <> Nullable()) 
    |> Seq.length

let l4 = lowPassFilteredSignal |> Seq.length
l0,ln,l1,l2,l3,l4, l3-l4

In [ ]:
let weatherDataMontreal = weatherDataMontreal - 

In [ ]:
let roll = 
    DataFrame.LoadCsv(
        filename=(@"C:\Users\cernu\Source\Repos\DS-AI\TakensTheorem\TakensTheorem.Test\expected\MontrealRollingWindow24Mean.csv" :> string), 
        dataTypes=[|typeof<string>;typeof<single>|],
        columnNames=[|"datetime";"Montreal"|],
        header=true)
    
roll.Columns

In [ ]:
Array.prepend

In [ ]:
[1..15] |> Seq.windowed 5 |> Seq.map (printfn "%A") |> Array.ofSeq 

In [ ]:
[for h in [0. .. (16. - 1.)] do yield h.GetType().Name]

In [ ]:
let Plot<'T when 'T :> ValueType and 'T: struct and 'T: (new: unit -> 'T)>(source: PrimitiveDataFrameColumn<'T>) =
        Scatter(y = Seq.cast<'T> source) |> Chart.Plot

In [18]:
let (!>) (source: DataFrameColumn) = source :?> PrimitiveDataFrameColumn<_>

In [6]:
//let result : PrimitiveDataFrameColumn<float> =  !! temperatureDF.["Montreal"] //|> Plot<float>
let temp = DataFrame.LoadCsv(@"C:\Users\cernu\Source\Repos\DS-AI\TakensTheorem\TakensTheorem.Test\data\temperature.csv")

In [13]:
let Values<'T when 'T : (new : unit -> 'T) and 'T : struct and 'T :> ValueType> (source: PrimitiveDataFrameColumn<'T>)= 
    source |> Seq.cast<Nullable<'T>> |> Array.ofSeq

In [21]:
let k : Nullable<single>[] = !> temp.["Montreal"] |> Values<single>

In [24]:
k |> Seq.take 10 |> Seq.iter (printfn "%A")

<null>
285.8299866f
285.8346558f
285.8477783f
285.8609314f
285.874054f
285.887207f
285.9003601f
285.9134827f
285.9266357f


In [ ]:
e